In [1]:
import pandas as pd
import numpy as np

import lightgbm as lgb

from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold, ParameterGrid
from sklearn.metrics import median_absolute_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_selection import SelectKBest, f_classif

import itertools

from tqdm import tqdm
import pickle

In [42]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
anime_selected = pd.read_csv('data/anime_selected.csv')

/opt/conda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7,25,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
anime_synopsis_emb = pd.read_csv('data/anime_synopsis_emb_2.csv')

In [6]:
rating_selected = pd.read_csv('data/rating_selected.csv')

In [7]:
lightfm_emb_df = pd.read_csv('data/lightfm_emb_df_50e.csv')

In [8]:
anime_selected.head()

,anime_id,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,Genres_sep,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Hentai,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Unknown,Vampire,Yaoi,Yuri,Start air
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,Original,24 min. per ep.,R - 17+ (violence & profanity),28.0,39,1251960,61971,105808,718161,71513,26678,329800,229170,182126.0,131625.0,62330,20688,8904,3184.0,1357.0,741.0,1580.0,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,90s
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,"Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,Original,1 hr. 55 min.,R - 17+ (violence & profanity),159.0,518,273145,1174,4143,208333,1935,770,57964,30043,49201.0,49505.0,22632,5805,1877,577.0,221.0,109.0,379.0,"['Action', 'Drama', 'Mystery', 'Sci-Fi', 'Space']",1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,00s
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,Victor Entertainment,"Funimation, Geneon Entertainment USA",Madhouse,Manga,24 min. per ep.,PG-13 - Teens 13 or older,266.0,201,558913,12944,29113,343492,25465,13925,146918,50229,75651.0,86142.0,49432,15376,5838,1965.0,664.0,316.0,533.0,"['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'D...",1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,90s
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,"TV Tokyo, Bandai Visual, Dentsu, Victor Entert...","Funimation, Bandai Entertainment",Sunrise,Original,25 min. per ep.,PG-13 - Teens 13 or older,2481.0,1467,94683,587,4300,46165,5121,5378,33719,2182,4806.0,10128.0,11618,5709,2920,1083.0,353.0,164.0,131.0,"['Action', 'Mystery', 'Police', 'Supernatural'...",1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,00s
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,"TV Tokyo, Dentsu",Unknown,Toei Animation,Manga,23 min. per ep.,PG - Children,3710.0,4369,13224,18,642,7314,766,1108,3394,312,529.0,1242.0,1713,1068,634,265.0,83.0,50.0,27.0,"['Adventure', 'Fantasy', 'Shounen', 'Supernatu...",0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,00s


In [9]:
anime_synopsis_emb.head()

,anime_id,emb0,emb1,emb2,emb3,emb4,emb5,emb6,emb7,emb8,emb9,emb10,emb11,emb12,emb13,emb14,emb15,emb16,emb17,emb18,emb19,emb20,emb21,emb22,emb23,emb24,emb25,emb26,emb27,emb28,emb29,emb30,emb31,emb32,emb33,emb34,emb35,emb36,emb37,emb38,emb39,emb40,emb41,emb42,emb43,emb44,emb45,emb46,emb47,emb48,emb49,emb50,emb51,emb52,emb53,emb54,emb55,emb56,emb57,emb58,emb59,emb60,emb61,emb62,emb63,emb64,emb65,emb66,emb67,emb68,emb69,emb70,emb71,emb72,emb73,emb74,emb75,emb76,emb77,emb78,emb79,emb80,emb81,emb82,emb83,emb84,emb85,emb86,emb87,emb88,emb89,emb90,emb91,emb92,emb93,emb94,emb95,emb96,emb97,emb98,emb99,emb100,emb101,emb102,emb103,emb104,emb105,emb106,emb107,emb108,emb109,emb110,emb111,emb112,emb113,emb114,emb115,emb116,emb117,emb118,emb119,emb120,emb121,emb122,emb123,emb124,emb125,emb126,emb127,emb128,emb129,emb130,emb131,emb132,emb133,emb134,emb135,emb136,emb137,emb138,emb139,emb140,emb141,emb142,emb143,emb144,emb145,emb146,emb147,emb148,emb149,emb150,emb151,emb152,emb153,emb154,emb155,emb156,emb157,emb158,emb159,emb160,emb161,emb162,emb163,emb164,emb165,emb166,emb167,emb168,emb169,emb170,emb171,emb172,emb173,emb174,emb175,emb176,emb177,emb178,emb179,emb180,emb181,emb182,emb183,emb184,emb185,emb186,emb187,emb188,emb189,emb190,emb191,emb192,emb193,emb194,emb195,emb196,emb197,emb198,emb199,emb200,emb201,emb202,emb203,emb204,emb205,emb206,emb207,emb208,emb209,emb210,emb211,emb212,emb213,emb214,emb215,emb216,emb217,emb218,emb219,emb220,emb221,emb222,emb223,emb224,emb225,emb226,emb227,emb228,emb229,emb230,emb231,emb232,emb233,emb234,emb235,emb236,emb237,emb238,emb239,emb240,emb241,emb242,emb243,emb244,emb245,emb246,emb247,emb248,emb249,emb250,emb251,emb252,emb253,emb254,emb255,emb256,emb257,emb258,emb259,emb260,emb261,emb262,emb263,emb264,emb265,emb266,emb267,emb268,emb269,emb270,emb271,emb272,emb273,emb274,emb275,emb276,emb277,emb278,emb279,emb280,emb281,emb282,emb283,emb284,emb285,emb286,emb287,emb288,emb289,emb290,emb291,emb292,emb293,emb294,emb295,emb296,emb297,emb298,emb299,emb300,emb301,emb302,emb303,emb304,emb305,emb306,emb307,emb308,emb309,emb310,emb311,emb312,emb313,emb314,emb315,emb316,emb317,emb318,emb319,emb320,emb321,emb322,emb323,emb324,emb325,emb326,emb327,emb328,emb329,emb330,emb331,emb332,emb333,emb334,emb335,emb336,emb337,emb338,emb339,emb340,emb341,emb342,emb343,emb344,emb345,emb346,emb347,emb348,emb349,emb350,emb351,emb352,emb353,emb354,emb355,emb356,emb357,emb358,emb359,emb360,emb361,emb362,emb363,emb364,emb365,emb366,emb367,emb368,emb369,emb370,emb371,emb372,emb373,emb374,emb375,emb376,emb377,emb378,emb379,emb380,emb381,emb382,emb383
0,1,-0.037959,0.003580,-0.017211,-0.026130,0.019456,-0.019987,0.022723,0.017017,-0.028059,-0.007092,0.012273,0.008578,0.013832,-0.006762,-0.042466,0.034519,0.004619,-0.014714,0.019883,0.016247,-0.050487,-0.004393,-0.008996,0.030360,-0.061954,-0.003686,0.004443,0.035934,-0.009892,-0.010420,-0.007989,0.024069,-0.003625,-0.000580,0.024303,0.060942,0.007046,0.016987,-0.006914,-0.007529,0.011035,-0.051239,0.037628,0.021332,-0.010090,-0.040340,-0.036259,-0.006836,-0.017065,0.002348,0.015120,-0.063076,0.010560,-0.032255,0.017137,-0.059919,0.000420,0.014235,-0.014135,-0.043292,-0.007693,-0.020321,0.002422,0.018317,0.006145,-0.013301,-0.009228,0.014055,-0.020301,-0.025373,0.002097,0.003052,-0.004086,0.021696,0.007335,0.051250,0.020829,0.000427,-0.016291,-0.038543,0.024526,-0.027519,-0.041594,-0.024013,-0.007039,-0.022419,0.005931,-0.006159,0.009990,0.031608,-0.056912,0.019725,0.076017,-0.003284,-0.016677,0.045867,0.035525,-0.031557,-0.011693,0.066680,0.020487,-0.014247,-0.002489,0.013377,-0.023016,-0.039304,0.031196,-0.006577,0.041985,-0.061005,-0.034111,0.008960,-0.010846,0.046408,0.040058,-0.004427,0.007366,0.054954,-0.071297,-0.029499,0.069899,0.045632,0.021967,0.012959,0.003102,-0.017090,-0.028539,-0.015805,0.016950,-0.034482,-0.015134,0.000362,0.024712,-0.009196,-0.036359,0.014683,-0.040213,0.025584,-0.007407,-0.034642,0.032497,0.027439,-0.005449,-0.026949,-0.051447,-0.031572,0.017978,-0.01402

In [10]:
True in anime_synopsis_emb.isna().any()

False

In [11]:
rating_selected.head()

,user_id,anime_id,rating
0,0,430,9
1,0,1004,5
2,0,3010,7
3,0,570,7
4,0,2762,9


In [12]:
lightfm_emb_df.head()

,anime_id,lightfmemb0,lightfmemb1,lightfmemb2,lightfmemb3,lightfmemb4,lightfmemb5,lightfmemb6,lightfmemb7,lightfmemb8,lightfmemb9,lightfmemb10,lightfmemb11,lightfmemb12,lightfmemb13,lightfmemb14,lightfmemb15,lightfmemb16,lightfmemb17,lightfmemb18,lightfmemb19,lightfmemb20,lightfmemb21,lightfmemb22,lightfmemb23,lightfmemb24,lightfmemb25,lightfmemb26,lightfmemb27,lightfmemb28,lightfmemb29,lightfmemb30,lightfmemb31,lightfmemb32,lightfmemb33,lightfmemb34,lightfmemb35,lightfmemb36,lightfmemb37,lightfmemb38,lightfmemb39,lightfmemb40,lightfmemb41,lightfmemb42,lightfmemb43,lightfmemb44,lightfmemb45,lightfmemb46,lightfmemb47,lightfmemb48,lightfmemb49,lightfmemb50,lightfmemb51,lightfmemb52,lightfmemb53,lightfmemb54,lightfmemb55,lightfmemb56,lightfmemb57,lightfmemb58,lightfmemb59,lightfmemb60,lightfmemb61,lightfmemb62,lightfmemb63
0,1,0.877783,0.216084,-0.503182,0.437487,0.054265,-0.824538,-0.622960,0.182473,-0.178679,-0.337158,0.521606,-0.549872,0.118328,-0.137754,0.410715,-0.180973,-0.570817,-0.862738,0.190253,0.391911,-1.206864,-0.376387,-0.502265,-0.220076,0.133199,0.218592,0.131855,0.518180,-0.287929,0.397931,-0.149840,-0.199122,-1.612970,-0.434727,-0.474510,1.269207,0.567623,-0.342300,-0.433264,0.279489,0.713963,0.378506,-0.007666,0.115169,-0.089850,-0.549659,-0.761420,-1.464966,-1.322069,-0.032957,-1.080959,-0.135092,-1.022020,-0.002554,-0.664350,1.142069,-0.472538,0.082486,-1.632876,-0.314429,-0.641308,0.009389,-0.651457,-0.240969
1,5,0.788837,-0.012186,-0.317833,0.314195,-0.047927,-0.258797,-0.414207,0.587654,0.290944,-0.492094,0.504216,-0.563243,0.316588,-0.221832,0.152622,-0.144501,-0.525736,-0.729610,0.249949,0.364802,-0.320432,-0.059109,-0.257882,-0.179054,0.331877,0.272099,0.241218,0.240626,-0.176915,0.388873,0.232570,0.054042,-0.909219,-0.864141,-0.413376,0.867266,0.163863,-0.675938,-0.288388,0.368793,0.560334,0.124126,-0.081069,-0.065063,-0.163645,-0.172389,0.004206,-0.851234,-1.313750,-0.196758,-0.470921,-0.179368,-0.767311,0.157949,-0.861186,0.856685,-0.251793,0.220200,-1.072088,-0.158940,-0.560269,0.128557,-0.165758,-0.639258
2,6,0.868641,0.247856,-0.517894,0.442624,0.064535,-0.810399,-0.361278,0.194449,0.122730,-0.322278,0.711811,-0.660453,0.076834,0.134945,0.468751,-0.225444,-0.431001,-0.651314,-0.162654,0.598493,-0.790972,-0.237844,-0.007045,0.141971,0.236932,0.732294,-0.201798,0.245262,-0.111632,0.341748,0.124955,-0.436397,-1.282260,-0.007148,-0.567485,1.175146,0.732044,0.149928,-0.422363,0.668099,0.613865,0.190118,0.362475,0.072596,-0.187930,-0.255868,-0.827494,-1.074873,-1.356851,0.251918,-0.932196,-0.277612,-0.571510,0.198246,-0.286605,0.608716,-0.230222,-0.109961,-1.065986,-0.419032,-0.337330,0.226134,-0.690568,-0.468791
3,7,0.546605,0.087473,-0.639145,0.283394,0.253443,-0.820313,0.015405,0.474382,0.302539,-0.800443,0.892969,-0.388261,0.277505,-0.042568,0.439123,-0.364895,-0.849158,-0.788575,0.695727,-0.200876,-0.219717,0.442171,-0.076308,0.130964,0.399064,0.634226,0.026199,0.226828,-0.177381,0.274950,-0.100955,-0.826547,-0.780638,-0.124821,0.035293,0.996021,0.628169,0.184121,-0.471690,0.642785,0.278115,0.660980,0.575491,0.364003,-0.893204,-0.185376,-1.167023,-0.783266,-1.177665,0.000044,-0.797066,0.242582,-0.447440,0.599809,-0.748350,0.321003,-0.302048,-0.031326,-1.207911,-0.126790,-0.859715,0.039976,-0.823537,-0.901160
4,8,0.959497,0.375417,-0.479217,-0.081518,-0.138866,-0.301535,-0.927677,0.765599,-0.184445,-0.320570,0.921715,-0.276462,0.420868,0.263651,0.025486,0.253380,0.012840,-0.516066,0.260879,0.372026,-0.636345,-0.337303,-0.005577,0.332600,-0.262290,0.593170,-0.763047,0.121525,0.793068,0.211355,0.182324,-1.226115,-0.244048,-0.026609,0.078271,0.030275,0.350184,0.056900,-0.562028,0.689275,-0.038999,0.420757,0.326340,0.389768,-0.718323,-0.335632,-0.879263,-0.076213,-0.404206,0.182044,-0.542010,-0.604022,-0.492324,0.067137,-0.451040,0.034096,-0.833235,-1.011305,0.126696,-0.055475,0.057071,0.505040,-0.150823,-0.898033


In [13]:
True in lightfm_emb_df.isna().any()

False

In [14]:
rating_selected = rating_selected.merge(anime_selected[['anime_id', 'Genres']], on = 'anime_id', how = 'left')

In [15]:
rating_selected.head()

,user_id,anime_id,rating,Genres
0,0,430,9,"Military, Comedy, Historical, Drama, Fantasy, ..."
1,0,1004,5,"Drama, Psychological, Romance, Slice of Life"
2,0,3010,7,"Adventure, Historical, Shounen"
3,0,570,7,"Military, Police, Psychological, Drama, Romance"
4,0,2762,9,"Action, Adventure, Comedy, Romance, School, Sh..."


In [16]:
total_genres = sorted(set([i.strip() for i in itertools.chain.from_iterable([i.split(',') for i in anime_selected['Genres'].values])]))

In [17]:
len(total_genres)

44

In [18]:
def create_users(rating_selected):

    user_data = []

    for user_id in tqdm(rating_selected['user_id'].unique()):
        filter_user = rating_selected[rating_selected['user_id'] == user_id]
        total_anime_watch = len(filter_user)
        genres_user = filter_user['Genres']

        if isinstance(genres_user, str):
            genres_user = [genres_user]
        else:
            genres_user = genres_user.values

        user_gen_dat = {}

        all_genres = [x.strip() for x in (','.join(map(lambda x: str(x), genres_user))).split(',')]

        for unique_gen in total_genres:
            user_gen_dat[unique_gen] = all_genres.count(unique_gen)/total_anime_watch
        
        user_gen_dat['mean_rating'] = filter_user['rating'].sum()/total_anime_watch
        
        user_data.append({user_id: user_gen_dat})
        
    users = pd.DataFrame(user_data).stack().apply(pd.Series).reset_index().rename(columns = {'level_1': 'user_id'}).drop('level_0', axis = 1)
    
    return users

In [19]:
users = create_users(rating_selected)

100%|██████████| 13102/13102 [00:54<00:00, 239.84it/s]


In [20]:
users.head()

,user_id,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Hentai,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Unknown,Vampire,Yaoi,Yuri,mean_rating
0,0,0.285714,0.514286,0.000000,0.457143,0.000000,0.000000,0.428571,0.028571,0.314286,0.028571,0.000000,0.000000,0.171429,0.057143,0.000000,0.000000,0.114286,0.085714,0.000000,0.114286,0.028571,0.057143,0.028571,0.028571,0.057143,0.371429,0.028571,0.057143,0.114286,0.000000,0.085714,0.000000,0.342857,0.028571,0.114286,0.000000,0.028571,0.171429,0.200000,0.000000,0.0,0.057143,0.0,0.0,7.400000
1,1,0.427184,0.233010,0.000000,0.485437,0.000000,0.019417,0.368932,0.077670,0.223301,0.029126,0.009709,0.000000,0.029126,0.029126,0.000000,0.009709,0.038835,0.097087,0.029126,0.048544,0.009709,0.106796,0.038835,0.019417,0.087379,0.194175,0.000000,0.407767,0.135922,0.058252,0.000000,0.000000,0.524272,0.000000,0.087379,0.000000,0.174757,0.203883,0.213592,0.077670,0.0,0.009709,0.0,0.0,8.058252
2,2,0.568627,0.156863,0.000000,0.352941,0.000000,0.039216,0.294118,0.176471,0.078431,0.058824,0.117647,0.156863,0.000000,0.137255,0.000000,0.000000,0.019608,0.039216,0.058824,0.078431,0.019608,0.176471,0.039216,0.019608,0.196078,0.117647,0.000000,0.196078,0.196078,0.156863,0.000000,0.000000,0.450980,0.000000,0.039216,0.000000,0.078431,0.215686,0.313725,0.078431,0.0,0.039216,0.0,0.0,8.333333
3,3,0.377778,0.079365,0.025397,0.612698,0.009524,0.041270,0.219048,0.250794,0.152381,0.025397,0.209524,0.000000,0.015873,0.041270,0.003175,0.000000,0.069841,0.019048,0.041270,0.028571,0.006349,0.130159,0.050794,0.006349,0.076190,0.403175,0.009524,0.403175,0.146032,0.184127,0.022222,0.003175,0.196825,0.000000,0.212698,0.003175,0.025397,0.069841,0.295238,0.047619,0.0,0.031746,0.0,0.0,7.603175
4,4,0.271186,0.220339,0.000000,0.457627,0.000000,0.050847,0.584746,0.050847,0.330508,0.000000,0.050847,0.000000,0.110169,0.059322,0.050847,0.000000,0.194915,0.042373,0.093220,0.033898,0.042373,0.177966,0.016949,0.008475,0.059322,0.618644,0.000000,0.194915,0.127119,0.025424,0.347458,0.016949,0.177966,0.008475,0.237288,0.000000,0.008475,0.076271,0.288136,0.025424,0.0,0.033898,0.0,0.0,7.652542


In [21]:
lgb_dataset = rating_selected.copy()

In [22]:
lgb_dataset = lgb_dataset.merge(users[users.columns], on = 'user_id', how = 'left')

In [23]:
lgb_dataset = lgb_dataset.merge(lightfm_emb_df[lightfm_emb_df.columns], on = 'anime_id', how = 'left')

In [24]:
lgb_dataset = lgb_dataset.merge(anime_synopsis_emb[anime_synopsis_emb.columns], on = 'anime_id', how = 'left')

In [25]:
lgb_dataset.head()

,user_id,anime_id,rating,Genres,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Hentai,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Unknown,Vampire,Yaoi,Yuri,mean_rating,lightfmemb0,lightfmemb1,lightfmemb2,lightfmemb3,lightfmemb4,lightfmemb5,lightfmemb6,lightfmemb7,lightfmemb8,lightfmemb9,lightfmemb10,lightfmemb11,lightfmemb12,lightfmemb13,lightfmemb14,lightfmemb15,lightfmemb16,lightfmemb17,lightfmemb18,lightfmemb19,lightfmemb20,lightfmemb21,lightfmemb22,lightfmemb23,lightfmemb24,lightfmemb25,lightfmemb26,lightfmemb27,lightfmemb28,lightfmemb29,lightfmemb30,lightfmemb31,lightfmemb32,lightfmemb33,lightfmemb34,lightfmemb35,lightfmemb36,lightfmemb37,lightfmemb38,lightfmemb39,lightfmemb40,lightfmemb41,lightfmemb42,lightfmemb43,lightfmemb44,lightfmemb45,lightfmemb46,lightfmemb47,lightfmemb48,lightfmemb49,lightfmemb50,lightfmemb51,lightfmemb52,lightfmemb53,lightfmemb54,lightfmemb55,lightfmemb56,lightfmemb57,lightfmemb58,lightfmemb59,lightfmemb60,lightfmemb61,lightfmemb62,lightfmemb63,emb0,emb1,emb2,emb3,emb4,emb5,emb6,emb7,emb8,emb9,emb10,emb11,emb12,emb13,emb14,emb15,emb16,emb17,emb18,emb19,emb20,emb21,emb22,emb23,emb24,emb25,emb26,emb27,emb28,emb29,emb30,emb31,emb32,emb33,emb34,emb35,emb36,emb37,emb38,emb39,emb40,emb41,emb42,emb43,emb44,emb45,emb46,emb47,emb48,emb49,emb50,emb51,emb52,emb53,emb54,emb55,emb56,emb57,emb58,emb59,emb60,emb61,emb62,emb63,emb64,emb65,emb66,emb67,emb68,emb69,emb70,emb71,emb72,emb73,emb74,emb75,emb76,emb77,emb78,emb79,emb80,emb81,emb82,emb83,emb84,emb85,emb86,emb87,emb88,emb89,emb90,emb91,emb92,emb93,emb94,emb95,emb96,emb97,emb98,emb99,emb100,emb101,emb102,emb103,emb104,emb105,emb106,emb107,emb108,emb109,emb110,emb111,emb112,emb113,emb114,emb115,emb116,emb117,emb118,emb119,emb120,emb121,emb122,emb123,emb124,emb125,emb126,emb127,emb128,emb129,emb130,emb131,emb132,emb133,emb134,emb135,emb136,emb137,emb138,emb139,emb140,emb141,emb142,emb143,emb144,emb145,emb146,emb147,emb148,emb149,emb150,emb151,emb152,emb153,emb154,emb155,emb156,emb157,emb158,emb159,emb160,emb161,emb162,emb163,emb164,emb165,emb166,emb167,emb168,emb169,emb170,emb171,emb172,emb173,emb174,emb175,emb176,emb177,emb178,emb179,emb180,emb181,emb182,emb183,emb184,emb185,emb186,emb187,emb188,emb189,emb190,emb191,emb192,emb193,emb194,emb195,emb196,emb197,emb198,emb199,emb200,emb201,emb202,emb203,emb204,emb205,emb206,emb207,emb208,emb209,emb210,emb211,emb212,emb213,emb214,emb215,emb216,emb217,emb218,emb219,emb220,emb221,emb222,emb223,emb224,emb225,emb226,emb227,emb228,emb229,emb230,emb231,emb232,emb233,emb234,emb235,emb236,emb237,emb238,emb239,emb240,emb241,emb242,emb243,emb244,emb245,emb246,emb247,emb248,emb249,emb250,emb251,emb252,emb253,emb254,emb255,emb256,emb257,emb258,emb259,emb260,emb261,emb262,emb263,emb264,emb265,emb266,emb267,emb268,emb269,emb270,emb271,emb272,emb273,emb274,emb275,emb276,emb277,emb278,emb279,emb280,emb281,emb282,emb283,emb284,emb285,emb286,emb287,emb288,emb289,emb290,emb291,emb292,emb293,emb294,emb295,emb296,emb297,emb298,emb299,emb300,emb301,emb302,emb303,emb304,emb305,emb306,emb307,emb308,emb309,emb310,emb311,emb312,emb313,emb314,emb315,emb316,emb317,emb318,emb319,emb320,emb321,emb322,emb323,emb324,emb325,emb326,emb327,emb328,emb329,emb330,emb331,emb332,emb333,emb334,emb335,emb336,emb337,emb338,emb339,emb340,emb341,emb342,emb343,emb344,emb345,emb346,emb347,emb348,emb349,emb350,emb351,emb352,emb353,emb354,emb355,emb356,emb357,emb358,emb359,emb360,emb361,emb362,emb363,emb364,emb365,emb366,emb367,emb368,emb369,emb370,emb371,emb372,emb373,emb374,emb375,emb376,emb377,emb378,emb379,emb380,emb381,emb382,emb383
0,0,430,9,"Military, Comedy, Historical, Drama, Fantasy, ...",0.285714,0.514286,0.0,0.457143,0.0,0.0,0.428571,0.028571,0.314286,0.028571,0.0,0.0,0.171429,0.057143,0.0,0.0,0.114286,0.085714,0.0,0.114286,0.0285

In [26]:
user_id_train, user_id_test = train_test_split(lgb_dataset['user_id'].unique(), test_size = 0.2, random_state = 2023) 

In [27]:
user_id_train, user_id_val = train_test_split(user_id_train, test_size = 0.15, random_state = 586) 

In [28]:
print(f'Num train {len(user_id_train)}')
print(f'Num val {len(user_id_val)}')
print(f'Num test {len(user_id_test)}')

Num train 8908
Num val 1573
Num test 2621


In [29]:
train = lgb_dataset[lgb_dataset['user_id'].isin(user_id_train)]
val = lgb_dataset[lgb_dataset['user_id'].isin(user_id_val)]
test = lgb_dataset[lgb_dataset['user_id'].isin(user_id_test)]

In [30]:
lgb_train = lgb.Dataset(train.iloc[:, 4:], train['rating'])
lgb_val = lgb.Dataset(val.iloc[:, 4:], val['rating'])

In [31]:
params = {'objective': 'regression',
          'max_depth': 8,
          'n_estimators': 2000,
          'num_leaves': 2**8-1,
          'learning_rate': 0.01,
          'colsample_bytree': 0.8,
          'subsample': 0.8,
          'early_stopping_rounds': 20,
          'random_state': 42,
          'n_jobs': 8
         }

In [32]:
lgbm_regressor = lgb.train(params,
                           lgb_train,
                           valid_sets = lgb_val,
                           verbose_eval = 10
                          )

/opt/conda/envs/py36/lib/python3.6/site-packages/lightgbm/engine.py:116: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/envs/py36/lib/python3.6/site-packages/lightgbm/engine.py:121: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 20 rounds.
[10]	valid_0's l2: 2.70186
[20]	valid_0's l2: 2.53489
[30]	valid_0's l2: 2.41606
[40]	valid_0's l2: 2.31213
[50]	valid_0's l2: 2.20095
[60]	valid_0's l2: 2.12524
[70]	valid_0's l2: 2.05451
[80]	valid_0's l2: 1.98845
[90]	valid_0's l2: 1.93758
[100]	valid_0's l2: 1.8915
[110]	valid_0's l2: 1.85023
[120]	valid_0's l2: 1.81613
[130]	valid_0's l2: 1.78674
[140]	valid_0's l2: 1.75984
[150]	valid_0's l2: 1.73927
[160]	valid_0's l2: 1.71986
[170]	valid_0's l2: 1.70364
[180]	valid_0's l2: 1.68872
[190]	valid_0's l2: 1.67577
[200]	valid_0's l2: 1.66516
[210]	valid_0's l2: 1.65546
[220]	valid_0's l2: 1.64615
[230]	valid_0's l2: 1.63782
[240]	valid_0's l2: 1.63034
[250]	valid_0's l2: 1.62372
[260]	valid_0's l2: 1.61782
[270]	valid_0's l2: 1.61247
[280]	valid_0's l2: 1.60749
[290]	valid_0's l2: 1.60294
[300]	valid_0's l2: 1.59857
[310]	valid_0's l2: 1.5948
[320]	valid_0's l2: 1.59132
[330]	valid_0's l2: 1.58808
[340]	valid_0's l2: 1.584

In [33]:
test_preds = lgbm_regressor.predict(test.iloc[:, 4:])

In [34]:
median_absolute_error(test['rating'], test_preds) #0.7200622085525481 with params set 1

0.7200622085525481

In [37]:
# with open('/home/dmitry-ds/rec-sys/models/LightGBM-v5.pickle', 'wb') as file:
#     pickle.dump(lgbm_regressor, file)

In [ ]:
#predictions for user

In [ ]:
# def recommend_for_user(N):
    
#     user_N = lgb_dataset[lgb_dataset['user_id'] == N]
    
#     lgb_dataset_user_N = lgb_dataset.drop_duplicates(subset = 'anime_id')
#     lgb_dataset_user_N.iloc[:, 5:49] = user_N.iloc[0, 5:49].values
#     lgb_dataset_user_N['user_id'] = 77
    
#     preds_user_N = lgbm_regressor.predict(lgb_dataset_user_N.iloc[:, 4:])
#     lgb_dataset_user_N['predicted_rating'] = preds_user_N
    
#     watched = lgb_dataset_user_N[lgb_dataset_user_N['anime_id'].\
#                                  isin(user_N.anime_id.values)][['anime_id', 'predicted_rating']].\
#     merge(user_N[['anime_id', 'rating']], on = 'anime_id').\
#     merge(anime_selected[['anime_id', 'Name']], on = 'anime_id').\
#     sort_values(by = 'rating', ascending = False)
    
#     top_10_user = lgb_dataset_user_N[~lgb_dataset_user_N['anime_id'].\
#                                      isin(user_N.anime_id.values)][['anime_id', 'predicted_rating']].\
#     merge(anime_selected[['anime_id', 'Name']], on = 'anime_id').\
#     sort_values(by = 'predicted_rating', ascending = False).iloc[:10, :]
    
#     return watched, top_10_user

In [ ]:
#watched, recs = recommend_for_user(77)

In [ ]:
#watched, recs = recommend_for_user(8)

In [ ]:
#watched, recs = recommend_for_user(4050)

In [ ]:
#watched, recs = recommend_for_user(217)

In [ ]:
#watched, recs = recommend_for_user(3580)

In [158]:
#using cosine similarity

def recommend_for_user_cos(N):
    
    user_N = lgb_dataset[lgb_dataset['user_id'] == N]
    
    lgb_dataset_user_N = lgb_dataset.drop_duplicates(subset = 'anime_id')
    lgb_dataset_user_N.iloc[:, 5:49] = user_N.iloc[0, 5:49].values
    lgb_dataset_user_N['user_id'] = N
    
    preds_user_N = lgbm_regressor.predict(lgb_dataset_user_N.iloc[:, 4:])
    lgb_dataset_user_N['predicted_rating'] = preds_user_N
    
    emb_cols = [col for col in lgb_dataset_user_N.columns if 'emb' in col]
    
    watched = lgb_dataset_user_N[lgb_dataset_user_N['anime_id'].\
                                 isin(user_N.anime_id.values)][['anime_id', 'predicted_rating']+emb_cols].\
    merge(user_N[['anime_id', 'rating']], on = 'anime_id').\
    merge(anime_selected[['anime_id', 'Name']], on = 'anime_id').\
    sort_values(by = 'rating', ascending = False)
    
    user_favorite_emb = np.mean(watched[watched['rating'] == watched['rating'].max()][emb_cols].values, axis = 0)
    
    top_1000_user = lgb_dataset_user_N[~lgb_dataset_user_N['anime_id'].\
                                     isin(user_N.anime_id.values)][['anime_id', 'predicted_rating']+emb_cols].\
    merge(anime_selected[['anime_id', 'Name']], on = 'anime_id').\
    sort_values(by = 'predicted_rating', ascending = False).iloc[:1000, :]
    
    top_1000_user['score'] = cosine_similarity([user_favorite_emb], top_1000_user[emb_cols].values)[0]
    
    top_10_user = top_1000_user.sort_values(by = ['score', 'predicted_rating'], ascending = False).\
    iloc[:10, :][['anime_id', 'Name', 'predicted_rating', 'score']]
    
    top_10_user = top_10_user.sort_values(by = 'predicted_rating', ascending = False)
    
    watched = watched[['anime_id', 'Name', 'rating', 'predicted_rating']]
    
    return watched, top_10_user

In [159]:
watched, recs = recommend_for_user_cos(77)

In [160]:
watched

,anime_id,Name,rating,predicted_rating
17,322,Paradise Kiss,10,8.557047
12,9756,Mahou Shoujo Madoka★Magica,10,8.975450
2,19,Monster,10,9.547424
3,1575,Code Geass: Hangyaku no Lelouch,10,8.860159
4,5114,Fullmetal Alchemist: Brotherhood,10,9.242316
5,1535,Death Note,10,8.992652
6,10620,Mirai Nikki,10,7.719719
16,339,Serial Experiments Lain,10,8.889393
13,11981,Mahou Shoujo Madoka★Magica Movie 3: Hangyaku n...,10,8.965994
10,877,Nana,10,9.002111


In [161]:
recs

,anime_id,Name,predicted_rating,score
756,4181,Clannad: After Story,9.305163,0.861862
332,2251,Baccano!,9.177494,0.864979
182,205,Samurai Champloo,9.084284,0.855355
629,2001,Tengen Toppa Gurren Lagann,9.065481,0.893775
155,2904,Code Geass: Hangyaku no Lelouch R2,9.038518,0.857830
952,1210,NHK ni Youkoso!,9.027711,0.880369
696,934,Higurashi no Naku Koro ni,8.820800,0.851522
1172,6211,Tokyo Magnitude 8.0,8.714334,0.855871
153,28223,Death Parade,8.634341,0.859120
92,4224,Toradora!,8.629137,0.855600


In [162]:
watched, recs = recommend_for_user_cos(4050)

In [163]:
watched

,anime_id,Name,rating,predicted_rating
3,934,Higurashi no Naku Koro ni,10,8.902158
1,32828,Amaama to Inazuma,9,8.313545
2,22789,Barakamon,9,9.044329
4,21511,Kantai Collection: KanColle,8,7.825873
0,16592,Danganronpa: Kibou no Gakuen to Zetsubou no Ko...,7,7.782647


In [164]:
recs

,anime_id,Name,predicted_rating,score
51,19,Monster,9.483270,0.789292
766,4181,Clannad: After Story,9.378492,0.788428
83,1535,Death Note,9.053018,0.790921
702,1889,Higurashi no Naku Koro ni Kai,8.951049,0.965028
1800,339,Serial Experiments Lain,8.943627,0.820276
968,1530,Kanon (2006),8.834592,0.767815
318,2167,Clannad,8.724322,0.786107
966,2924,ef: A Tale of Memories.,8.710527,0.790614
61,6547,Angel Beats!,8.563849,0.777259
1251,7724,Shiki,8.552313,0.779236


In [165]:
watched, recs = recommend_for_user_cos(8)

In [166]:
watched

,anime_id,Name,rating,predicted_rating
0,578,Hotaru no Haka,9,8.399864
10,33222,Fudanshi Koukou Seikatsu,9,6.415823
15,10408,Hotarubi no Mori e,9,8.257022
13,31297,"Tokyo Ghoul: ""Pinto""",9,7.188359
7,30,Neon Genesis Evangelion,9,8.479655
12,4181,Clannad: After Story,9,8.808925
16,18671,Chuunibyou demo Koi ga Shitai! Ren,8,7.165471
14,31,Neon Genesis Evangelion: Death & Rebirth,8,7.163371
6,32093,Tanaka-kun wa Itsumo Kedaruge,8,7.648278
8,2167,Clannad,8,7.742593


In [167]:
recs

,anime_id,Name,predicted_rating,score
843,12355,Ookami Kodomo no Ame to Yuki,8.498759,0.857557
317,9989,Ano Hi Mita Hana no Namae wo Bokutachi wa Mada...,8.072534,0.834661
20,2236,Toki wo Kakeru Shoujo,8.041810,0.854818
1168,6211,Tokyo Magnitude 8.0,8.004730,0.835624
95,4224,Toradora!,7.961787,0.839216
225,16067,Nagi no Asu kara,7.927633,0.834818
60,6547,Angel Beats!,7.779224,0.881204
182,5681,Summer Wars,7.681586,0.841807
261,28297,Ore Monogatari!!,7.609806,0.843429
12,1689,Byousoku 5 Centimeter,7.478754,0.863336


In [168]:
watched, recs = recommend_for_user_cos(8954)

In [169]:
watched

,anime_id,Name,rating,predicted_rating
0,164,Mononoke Hime,10,9.393767
9,1454,Kemonozume,10,9.003533
17,1060,Oruchuban Ebichu,10,8.795229
16,1888,Ichigo Mashimaro OVA,10,8.904579
15,488,Ichigo Mashimaro,10,8.871418
12,320,A Kite,10,7.994135
19,1883,Kappa no Kaikata,10,8.324041
7,2164,Dennou Coil,9,9.132319
6,3323,Kite Liberator,9,8.209769
11,221,Mezzo Forte,9,8.108233


In [170]:
recs

,anime_id,Name,predicted_rating,score
187,205,Samurai Champloo,9.374145,0.787905
959,1210,NHK ni Youkoso!,9.239359,0.792997
1495,1033,Sennen Joyuu,9.210520,0.792442
1504,759,Tokyo Godfathers,9.197010,0.805356
2171,387,Haibane Renmei,9.022858,0.804754
339,66,Azumanga Daioh,8.969665,0.857315
301,240,Genshiken,8.935716,0.805210
3309,3001,Moyashimon,8.906552,0.798810
893,1887,Lucky☆Star,8.895067,0.793703
3857,489,Kamichu!,8.883602,0.838897


In [171]:
watched, recs = recommend_for_user_cos(5000)

In [172]:
watched

,anime_id,Name,rating,predicted_rating
0,1535,Death Note,8,8.835509


In [173]:
recs

,anime_id,Name,predicted_rating,score
79,5114,Fullmetal Alchemist: Brotherhood,9.272708,0.858747
161,2904,Code Geass: Hangyaku no Lelouch R2,9.197164,0.825956
51,19,Monster,9.065621,0.807172
769,4181,Clannad: After Story,9.000059,0.800276
70,1575,Code Geass: Hangyaku no Lelouch,8.932051,0.854521
642,2001,Tengen Toppa Gurren Lagann,8.710935,0.800041
128,16498,Shingeki no Kyojin,8.661209,0.819281
97,4224,Toradora!,8.203709,0.801153
61,6547,Angel Beats!,8.146401,0.828375
645,3588,Soul Eater,7.901040,0.829399


In [176]:
watched, recs = recommend_for_user_cos(2686)

In [177]:
watched

,anime_id,Name,rating,predicted_rating
0,6347,Baka to Test to Shoukanjuu,9,8.999122


In [178]:
recs

,anime_id,Name,predicted_rating,score
124,13759,Sakura-sou no Pet na Kanojo,9.290104,0.806297
61,6547,Angel Beats!,9.284146,0.835822
288,11843,Danshi Koukousei no Nichijou,9.274542,0.856817
98,4224,Toradora!,9.257775,0.832100
797,72,Full Metal Panic? Fumoffu,9.213891,0.798952
1186,6213,Toaru Kagaku no Railgun,9.187587,0.811860
564,846,School Rumble Ni Gakki,9.155018,0.824928
420,15809,Hataraku Maou-sama!,9.112972,0.843658
476,849,Suzumiya Haruhi no Yuuutsu,9.107707,0.817325
434,10080,Kami nomi zo Shiru Sekai II,9.089796,0.816990


## Measure precision@10 and recall@10

In [ ]:
# Precision at k and recall at k are evaluation metrics for ranking algorithms. 
# Precision at k shows the share of relevant items in the first k results of the ranking algorithm. 
# And Recall at k indicates the share of relevant items returned in top k results out of all correct answers for a given query.

# Example: For a search query “Car” there are 3 relevant products in your shop. 
# Your search algorithm returns 2 of those relevant products in the first 5 search results. 
# Precision at 5 = # num of relevant products in search result / k = 2/5 = 40% 
# Recall at 5 = # num of relevant products in search result / # num of all relevant products = 2/3 = 66.6%

In [61]:
def get_metrics(N, num_rec = 10):
    
    user_N = lgb_dataset[lgb_dataset['user_id'] == N]

    lgb_dataset_user_N = lgb_dataset.drop_duplicates(subset = 'anime_id')
    lgb_dataset_user_N.iloc[:, 5:49] = user_N.iloc[0, 5:49].values
    lgb_dataset_user_N['user_id'] = N

    preds_user_N = lgbm_regressor.predict(lgb_dataset_user_N.iloc[:, 4:])
    lgb_dataset_user_N['predicted_rating'] = preds_user_N
    
    emb_cols = [col for col in lgb_dataset_user_N.columns if 'emb' in col]
    
    watched = lgb_dataset_user_N[lgb_dataset_user_N['anime_id'].\
                                     isin(user_N.anime_id.values)][['anime_id', 'predicted_rating']+emb_cols].\
    merge(user_N[['anime_id', 'rating']], on = 'anime_id').\
    merge(anime_selected[['anime_id', 'Name']], on = 'anime_id').\
    sort_values(by = ['rating'], ascending = False)
        
    user_favorite_emb = np.mean(watched[watched['rating'] == watched['rating'].max()][emb_cols].values, axis = 0)
    
    top_1000_user = lgb_dataset_user_N[['anime_id', 'predicted_rating']+emb_cols].\
    merge(anime_selected[['anime_id', 'Name']], on = 'anime_id').\
    sort_values(by = 'predicted_rating', ascending = False).iloc[:1000, :]

    top_1000_user['score'] = cosine_similarity([user_favorite_emb], top_1000_user[emb_cols].values)[0]

    top_user = top_1000_user.sort_values(by = ['score', 'predicted_rating'], ascending = False).\
    iloc[:num_rec, :][['anime_id', 'Name', 'predicted_rating', 'score']]

    top_user = top_user.sort_values(by = ['predicted_rating'], ascending = False)

    favs = watched[['anime_id', 'Name', 'rating', 'predicted_rating']][(watched['rating']==watched['rating'].max())|(watched['rating']==watched['rating'].max()-1)]
    
    num_common = len(list(set(favs['Name'].values).intersection(top_user['Name'].values)))
    
    precision = num_common/num_rec
    recall = num_common/len(favs)
    
    return precision, recall

In [62]:
precision_list = []
recall_list = []

for i in tqdm(user_id_test):
    precision, recall = get_metrics(i)
    precision_list.append(precision)
    recall_list.append(recall)

100%|██████████| 2621/2621 [1:50:26<00:00,  2.53s/it]


In [63]:
np.mean(precision_list)

0.44467760396795114

In [64]:
np.mean(recall_list)

0.1860742432591797